<a href="https://colab.research.google.com/github/YunSeoHwan/DNN_Study/blob/main/C4_CNN/VGG_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **VGG Classification**

# **Define Block**

In [1]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):

    # define layer
    def __init__(self, in_channels, out_channels, hidden_dim):
        # 상속 요소
        super(BasicBlock, self).__init__()

        # define conv layer
        self.conv1 = nn.Conv2d(in_channels, hidden_dim,
                               kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, out_channels,
                               kernel_size=3, padding=1)
        self.relu = nn.ReLU()

        # pooling, stride -> size 절반
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    
    # define 순전파
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)

        return x

# **Define Model**

In [2]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        
        super(CNN, self).__init__()
    
        # 합성곱 기본 블록 정의
        self.block1 = BasicBlock(in_channels=3, out_channels=32, hidden_dim=16)
        self.block2 = BasicBlock(in_channels=32, out_channels=128, hidden_dim=64)
        self.block3 = BasicBlock(in_channels=128, out_channels=256, hidden_dim=128)

        # fully connected
        self.fc1 = nn.Linear(in_features=4096, out_features=2048)       # 4 x 4 x 256 = 4096
        self.fc2 = nn.Linear(in_features=2048, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=num_classes)

        # activation function
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)                  # (-1, 256, 4, 4) -> batch는 무시
        x = torch.flatten(x, start_dim=1)   # 1차원으로

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        return x

# **Data Load & Define Model**

In [3]:
from torch.cuda import device_count

from torchvision.transforms import Compose
from torchvision.transforms import RandomHorizontalFlip, RandomCrop
from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
from torchvision.transforms import Normalize
from torchvision.datasets.cifar import CIFAR10
from torchvision.transforms import ToTensor

transforms = Compose([
    RandomCrop((32, 32), padding=4),
    RandomHorizontalFlip(p=0.5),
    ToTensor(),
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.247, 0.243, 0.261))
])

# data load
training_data = CIFAR10(
    root="./",
    train=True,
    download=True,
    transform=transforms)

test_data = CIFAR10(
    root="./",
    train=False,
    download=True,
    transform=transforms)

# define loader
train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# processor
device = "cuda" if torch.cuda.is_available() else "cpu"

# define model
model = CNN(num_classes=10)

model.to(device)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


CNN(
  (block1): BasicBlock(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): BasicBlock(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu): ReLU()
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4096, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=256,

# **Training**

In [4]:
# learning_rate
lr = 1e-3

# optimization
optim = Adam(model.parameters(), lr=lr)

for epoch in range(100):
    for data, label in train_loader:

        # 기울기 초기화
        optim.zero_grad()

        # predict
        pred = model(data.to(device))

        # backpropagation
        loss = nn.CrossEntropyLoss()(pred, label.to(device))
        loss.backward()
        optim.step()
    
    if epoch == 0 or epoch % 10 == 9:
        print(f"epoch{epoch + 1} loss : {loss.item()}")

# save
torch.save(model.state_dict(), "CIFAR.pth")

epoch1 loss : 1.177912950515747
epoch10 loss : 0.5645318627357483
epoch20 loss : 0.279543936252594
epoch30 loss : 1.3675956726074219
epoch40 loss : 0.576065182685852
epoch50 loss : 0.2819298505783081
epoch60 loss : 0.39476504921913147
epoch70 loss : 1.0638748407363892
epoch80 loss : 0.6258436441421509
epoch90 loss : 0.5969761610031128
epoch100 loss : 0.5790769457817078


# **Predict**

In [5]:
model.load_state_dict(torch.load("CIFAR.pth", map_location=device))

num_corr = 0

with torch.no_grad():
    for data, label in test_loader:

        output = model(data.to(device))
        pred = output.data.max(1)[1]
        corr = pred.eq(label.to(device).data).sum().item()
        num_corr += corr
    
    print(f"Accuracy : {num_corr / len(test_data)}")

Accuracy : 0.8178
